# Setup Vector DB

In [ ]:
# Copyright 2024 Drengskapur
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#
# @title {display-mode:"form"}
# @markdown <br/><br/><center><img src="https://cdn.jsdelivr.net/gh/drengskapur/docker-in-colab/assets/docker.svg" height="150"><img src="https://cdn.jsdelivr.net/gh/drengskapur/docker-in-colab/assets/colab.svg" height="150"></center><br/>
# @markdown <center><h1>Docker in Colab</h1></center><center>github.com/drengskapur/docker-in-colab<br/><br/><br/><b>udocker("run hello-world")</b></center><br/>
def udocker_init():
    import os
    if not os.path.exists("/home/user"):
        !pip install udocker > /dev/null
        !udocker --allow-root install > /dev/null
        !useradd -m user > /dev/null
    print(f'Docker-in-Colab 1.1.0\n')
    print(f'Usage:     udocker("--help")')
    print(f'Examples:  https://github.com/indigo-dc/udocker?tab=readme-ov-file#examples')

    def execute(command: str):
        user_prompt = "\033[1;32muser@pc\033[0m"
        print(f"{user_prompt}$ udocker {command}")
        !su - user -c "udocker $command"

    return execute

udocker = udocker_init()

Docker-in-Colab 1.1.0

Usage:     udocker("--help")
Examples:  https://github.com/indigo-dc/udocker?tab=readme-ov-file#examples


In [ ]:
!udocker --allow-root install

In [ ]:
!udocker --allow-root pull rubythalib/pgvector

Info: downloading layer sha256:90a681ec8319fc2e2f03d157d2733b6916a043d1dbc5832ae302d0736b50b972
Info: downloading layer sha256:3e4519aebcb16321f5b7b59092dfbcd7a01fb7e1bb8152536b94675730cb6191
Info: downloading layer sha256:135490726673bcd01a3e3e31fc6ff9db9296c69fea13e70331a7edf63895eb49
Info: downloading layer sha256:5ae8c52f27323035d588de8dfb1c943442bd741da263b49ae02dac4ac8f5c78d
Info: downloading layer sha256:6fa3ed8d50472d219b0a54f2a27594e240fcba78705c16737d3eda4e4053fd22
Info: downloading layer sha256:4f8b906dc57f465d053a04ed809cfa8d5fab25f4357707e9ed6b375d7696c901
Info: downloading layer sha256:e8a9c6f88b3ac0d34a58c6cacd68ece1c4d7f53b0e1d7017edead031d087050f
Info: downloading layer sha256:bd55a69456f7dbeefa051cdfb51f20e3917466e404530f6e793c5d59c6a27798
Info: downloading layer sha256:0a8727713246ed1e93b83ea6566473f0e49d331d6bd78d495a25921149423e7f
Info: downloading layer sha256:876fecb4c432d062ffcc214cbdec029ffe9455f30f3e591f5c9680322f52c963
Info: downloading layer sha256:c7813914a

In [ ]:
!udocker --allow-root images

REPOSITORY
rubythalib/pgvector:latest    .


In [ ]:
!nohup udocker --allow-root run -p 5432:5432 -e POSTGRES_PASSWORD=example rubythalib/pgvector > pgvector.log 2>&1 &

In [ ]:
!pip install psycopg2-binary psycopg2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 13.9 MB/s eta 0:00:00


In [ ]:
import psycopg2
from psycopg2 import sql

# Database credentials
DB_NAME = "llm_on_pro"
DB_USER = "postgres"
DB_PASSWORD = "example"
DB_HOST = "localhost"
DB_PORT = "5432"

# Function to create the database and tables
def setup_database_and_tables():
    # Step 1: Connect to PostgreSQL to create the database if it doesn't exist
    try:
        conn = psycopg2.connect(dbname="postgres", user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT)
        conn.autocommit = True  # Required for database creation
        cursor = conn.cursor()

        # Check if the database already exists
        cursor.execute("SELECT 1 FROM pg_database WHERE datname = %s", (DB_NAME,))
        exists = cursor.fetchone()

        # Create the database if it doesn't exist
        if not exists:
            cursor.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(DB_NAME)))
            print(f"Database '{DB_NAME}' created successfully.")
        else:
            print(f"Database '{DB_NAME}' already exists.")

        cursor.close()
        conn.close()
    except Exception as e:
        print(f"Error creating database: {e}")
        return

    # Step 2: Connect to the created database and create the tables if they don't exist
    try:
        conn = psycopg2.connect(dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT)
        cursor = conn.cursor()

        # Create cached_chat table if it doesn't exist
        create_cached_chat_table = """
        CREATE TABLE IF NOT EXISTS cached_chat (
            id SERIAL PRIMARY KEY,
            instruction TEXT NOT NULL,
            input_data TEXT NOT NULL,
            response TEXT NOT NULL,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
        """
        cursor.execute(create_cached_chat_table)

        # Create chat_history table if it doesn't exist
        create_chat_history_table = """
        CREATE TABLE IF NOT EXISTS chat_history (
            id SERIAL PRIMARY KEY,
            chat TEXT NOT NULL,
            answer TEXT NOT NULL,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
        """
        cursor.execute(create_chat_history_table)

        # Create analytics table if it doesn't exist
        create_analytics_table = """
        CREATE TABLE IF NOT EXISTS analytics (
            id SERIAL PRIMARY KEY,
            type VARCHAR(50) NOT NULL,  -- 'like', 'dislike', 'regenerate'
            chat_history_id INT REFERENCES chat_history(id) ON DELETE CASCADE,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
        """
        cursor.execute(create_analytics_table)

        conn.commit()
        print("Tables 'cached_chat', 'chat_history', and 'analytics' created or already exist.")

        cursor.close()
        conn.close()
    except Exception as e:
        print(f"Error creating tables: {e}")

# Call the function to setup the database and tables
setup_database_and_tables()

Database 'llm_on_pro' created successfully.
Tables 'cached_chat', 'chat_history', and 'analytics' created or already exist.


In [ ]:
def display_database_contents():
    """
    Connects to the database and displays the tables and their contents.
    """
    try:
        conn = psycopg2.connect(dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT)
        cursor = conn.cursor()

        # Get all table names
        cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public'")
        tables = [row[0] for row in cursor.fetchall()]

        if not tables:
            print("No tables found in the database.")
            return

        print(f"Tables in database '{DB_NAME}': {', '.join(tables)}")

        for table_name in tables:
            print(f"\n--- Contents of table: {table_name} ---")

            # Use sql.Identifier to prevent SQL injection
            cursor.execute(sql.SQL("SELECT * FROM {}").format(sql.Identifier(table_name)))
            rows = cursor.fetchall()

            if not rows:
                print("No data in this table.")
            else:
                # Get column names
                columns = [desc[0] for desc in cursor.description]
                print(f"{' | '.join(columns)}")
                for row in rows:
                    # Convert each value to string and join them
                    row_str = ' | '.join(map(str, row))
                    print(row_str)

        cursor.close()
        conn.close()
    except Exception as e:
        print(f"Error displaying database contents: {e}")

# Call the function to display the database contents
display_database_contents()

Tables in database 'llm_on_pro': cached_chat, chat_history, analytics

--- Contents of table: cached_chat ---
No data in this table.

--- Contents of table: chat_history ---
No data in this table.

--- Contents of table: analytics ---
No data in this table.


# Setup LLM

In [ ]:
!pip install fastapi uvicorn pyngrok nest-asyncio psycopg2-binary llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 MB 12.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.0 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.9-cp311-cp311-linux_x86_64.whl size=4067775 sha256=eb5cd2bd8cba28bd12d882dff50a973432f0d81341373d538c6f149b5a3e13a4
  Stored in directory: /root/.cache/pip/wheels/9e/8f/bf/148c8eb7d69021eccd6eae6444f3accd48347587054ffd24e5
Successfully built llama-cpp-python


In [ ]:
from fastapi import FastAPI, HTTPException, Query
from pydantic import BaseModel
from llama_cpp import Llama
import psycopg2
from psycopg2.extras import RealDictCursor
from typing import Optional
from datetime import datetime
from pyngrok import ngrok
import nest_asyncio
import uvicorn
from threading import Thread

In [ ]:
# Allow nested asyncio event loops for Colab
nest_asyncio.apply()

# Initialize FastAPI
app = FastAPI()

# Load Llama model (Assuming model is present or downloaded separately)
llm = Llama.from_pretrained(
    repo_id="rubythalib33/llama3_1_8b_finetuned_bahasa_indonesia",
    filename="unsloth.Q4_K_M.gguf",
)

# Update with your PostgreSQL connection info if using external DB
DB_NAME = "llm_on_pro"
DB_USER = "postgres"
DB_PASSWORD = "example"
DB_HOST = "localhost"
DB_PORT = "5432"

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


unsloth.Q4_K_M.gguf:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

llama_model_loader: loaded meta data with 27 key-value pairs and 292 tensors from /root/.cache/huggingface/hub/models--rubythalib33--llama3_1_8b_finetuned_bahasa_indonesia/snapshots/2f118fe76cf2dd5ebaba86fcebe460d99c2363d7/./unsloth.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Meta Llama 3.1 8b Bnb 4bit
llama_model_loader: - kv   3:                       general.organization str              = Unsloth
llama_model_loader: - kv   4:                           general.finetune str              = bnb-4bit
llama_model_loader: - kv   5:                           general.basename str              = meta-llama-3.1
llama_m

In [ ]:
DATABASE_URL = f"dbname={DB_NAME} user={DB_USER} password={DB_PASSWORD} host={DB_HOST} port={DB_PORT}"

class ChatRequest(BaseModel):
    instruction: str
    input_data: str = ""

class ChatResponse(BaseModel):
    response: str
    chat_history_id: int

class ReactionRequest(BaseModel):
    chat_history_id: int
    reaction: str

class RegenerateRequest(BaseModel):
    chat_history_id: int

alpaca_prompt = """Di bawah ini adalah instruksi yang menjelaskan tugas, dipasangkan dengan masukan yang memberikan konteks lebih lanjut. Tulis tanggapan yang melengkapi permintaan dengan tepat.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:
def get_cached_response(instruction: str, input_data: str):
    try:
        conn = psycopg2.connect(DATABASE_URL, cursor_factory=RealDictCursor)
        cursor = conn.cursor()
        cursor.execute("SELECT response FROM cached_chat WHERE instruction = %s AND input_data = %s", (instruction, input_data))
        result = cursor.fetchone()
        conn.close()
        return result
    except Exception as e:
        print(f"DB error (cache): {e}")
        return None

def cache_response(instruction: str, input_data: str, response: str):
    try:
        conn = psycopg2.connect(DATABASE_URL)
        cursor = conn.cursor()
        cursor.execute("INSERT INTO cached_chat (instruction, input_data, response) VALUES (%s, %s, %s)", (instruction, input_data, response))
        conn.commit()
        conn.close()
    except Exception as e:
        print(f"DB error (cache insert): {e}")

def save_chat_history(chat: str, answer: str):
    try:
        conn = psycopg2.connect(DATABASE_URL)
        cursor = conn.cursor()
        cursor.execute("INSERT INTO chat_history (chat, answer) VALUES (%s, %s) RETURNING id", (chat, answer))
        chat_history_id = cursor.fetchone()[0]
        conn.commit()
        conn.close()
        return chat_history_id
    except Exception as e:
        print(f"DB error (history): {e}")
        return None

def add_reaction(chat_history_id: int, reaction: str):
    try:
        conn = psycopg2.connect(DATABASE_URL)
        cursor = conn.cursor()
        cursor.execute("INSERT INTO analytics (type, chat_history_id) VALUES (%s, %s)", (reaction, chat_history_id))
        conn.commit()
        conn.close()
    except Exception as e:
        print(f"DB error (reaction): {e}")

def get_chat_history_by_id(chat_history_id: int):
    try:
        conn = psycopg2.connect(DATABASE_URL, cursor_factory=RealDictCursor)
        cursor = conn.cursor()
        cursor.execute("SELECT chat, answer FROM chat_history WHERE id = %s", (chat_history_id,))
        result = cursor.fetchone()
        conn.close()
        return result
    except Exception as e:
        print(f"DB error (history id): {e}")
        return None

def get_total_reactions(reaction_type: Optional[str], start_datetime: Optional[str], end_datetime: Optional[str]):
    try:
        conn = psycopg2.connect(DATABASE_URL, cursor_factory=RealDictCursor)
        cursor = conn.cursor()
        query = "SELECT COUNT(*) AS total FROM analytics WHERE 1=1"
        params = []

        if reaction_type:
            query += " AND type = %s"
            params.append(reaction_type)
        if start_datetime:
            query += " AND created_at >= %s"
            params.append(start_datetime)
        if end_datetime:
            query += " AND created_at <= %s"
            params.append(end_datetime)

        cursor.execute(query, params)
        result = cursor.fetchone()
        conn.close()
        return result["total"]
    except Exception as e:
        print(f"DB error (reactions): {e}")
        return None

In [ ]:
@app.post("/chat", response_model=ChatResponse)
async def chat_completion(request: ChatRequest):
    cached = get_cached_response(request.instruction, request.input_data)
    if cached:
        chat_history_id = save_chat_history(request.instruction, cached["response"])
        return ChatResponse(response=cached["response"], chat_history_id=chat_history_id)

    prompt = alpaca_prompt.format(request.instruction, request.input_data, "")
    result = llm.create_chat_completion(messages=[{"role": "user", "content": prompt}])
    response_text = result["choices"][0]["message"]["content"]

    cache_response(request.instruction, request.input_data, response_text)
    chat_history_id = save_chat_history(request.instruction, response_text)
    return ChatResponse(response=response_text, chat_history_id=chat_history_id)

@app.post("/regenerate", response_model=ChatResponse)
async def regenerate_chat(request: RegenerateRequest):
    history = get_chat_history_by_id(request.chat_history_id)
    if not history:
        raise HTTPException(status_code=404, detail="Chat history not found")

    prompt = alpaca_prompt.format(history["chat"], "", "")
    result = llm.create_chat_completion(messages=[{"role": "user", "content": prompt}])
    response_text = result["choices"][0]["message"]["content"]

    new_id = save_chat_history(history["chat"], response_text)
    add_reaction(request.chat_history_id, "regenerate")
    return ChatResponse(response=response_text, chat_history_id=new_id)

@app.post("/react")
async def react_to_chat(request: ReactionRequest):
    if request.reaction not in ["like", "dislike"]:
        raise HTTPException(status_code=400, detail="Invalid reaction.")
    add_reaction(request.chat_history_id, request.reaction)
    return {"message": "Reaction saved successfully."}

@app.get("/total-reactions")
async def get_total_reaction_count(
    reaction_type: Optional[str] = Query(None),
    start_datetime: Optional[str] = Query(None),
    end_datetime: Optional[str] = Query(None)
):
    total = get_total_reactions(reaction_type, start_datetime, end_datetime)
    if total is None:
        raise HTTPException(status_code=500, detail="Error retrieving reactions.")
    return {"total_reactions": total}

In [ ]:
# Authtoken from your ngrok account (replace this with your real token)
ngrok.set_auth_token("NGROK_KEY")

In [ ]:
# Expose the FastAPI server on port 8000
public_url = ngrok.connect(8000)
print("Public URL:", public_url)

Public URL: NgrokTunnel: "https://bd7b-34-169-138-129.ngrok-free.app" -> "http://localhost:8000"


In [ ]:
# Start FastAPI in a background thread
def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

thread = Thread(target=run)
thread.start()

# Setup Streamlit

In [ ]:
!pip install streamlit pyngrok pandas requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.0 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import requests
from datetime import datetime, timedelta

# Set default values
default_reaction_type = "dislike"
default_start_date = datetime.now() - timedelta(days=365)
default_end_date = datetime.now()

# Streamlit UI
st.title("Online Metrics Monitoring Dashboard")

reaction_type = st.selectbox(
    "Select Reaction Type:",
    options=["like", "dislike", "regenerate"],
    index=1  # default is "dislike"
)

start_date = st.date_input(
    "Select Start Date:",
    value=default_start_date
)

end_date = st.date_input(
    "Select End Date:",
    value=default_end_date
) + timedelta(days=1)

# Convert dates to string format
start_date_str = start_date.strftime("%Y-%m-%d %H:%M:%S")
end_date_str = end_date.strftime("%Y-%m-%d %H:%M:%S")

# Function to fetch reaction data from the FastAPI backend
def fetch_reaction_data(reaction_type, start_date, end_date):
    url = f"https://bd7b-34-169-138-129.ngrok-free.app/total-reactions"
    params = {
        "reaction_type": reaction_type,
        "start_datetime": start_date,
        "end_datetime": end_date
    }

    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            return data['total_reactions']
        else:
            st.error("Failed to fetch data from the API.")
            return None
    except Exception as e:
        st.error(f"Error: {e}")
        return None

# Fetch the data from FastAPI
reaction_count = fetch_reaction_data(reaction_type, start_date_str, end_date_str)

if reaction_count is not None:
    df = pd.DataFrame({
        'Date': pd.date_range(start=start_date, end=end_date, periods=30),
        'Reactions': [reaction_count for _ in range(30)]
    })
    st.line_chart(df.set_index('Date'))

Writing app.py


In [ ]:
from pyngrok import ngrok
import threading
import time
import os

# Jalankan Streamlit di background
def run_streamlit():
    os.system("streamlit run app.py")

# Jalankan streamlit di thread terpisah
threading.Thread(target=run_streamlit).start()

# Tunggu beberapa detik agar Streamlit bisa jalan
time.sleep(5)

# Buat tunnel ngrok ke port 8501
public_url = ngrok.connect(8501)
print(f"Streamlit is live at: {public_url}")

Streamlit is live at: NgrokTunnel: "https://2f77-34-169-138-129.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
display_database_contents()

Tables in database 'llm_on_pro': cached_chat, chat_history, analytics

--- Contents of table: cached_chat ---
id | instruction | input_data | response | created_at
1 | Siapa presiden pertama Indonesia |  | Presiden pertama Indonesia adalah Soekarno, yang menjabat dari tahun 1945 hingga 1966. | 2025-05-19 14:27:01.913025
2 | Berapa jumlah hari dalam seminggu |  |  Ada 7 hari dalam seminggu. | 2025-05-19 14:35:41.754259

--- Contents of table: chat_history ---
id | chat | answer | created_at
1 | Siapa presiden pertama Indonesia | Presiden pertama Indonesia adalah Soekarno, yang menjabat dari tahun 1945 hingga 1966. | 2025-05-19 14:27:01.940343
2 | Siapa presiden pertama Indonesia | Presiden pertama Indonesia adalah Soekarno, yang menjabat dari tahun 1945 hingga tahun 1967. | 2025-05-19 14:31:39.625971
3 | Berapa jumlah hari dalam seminggu |  Ada 7 hari dalam seminggu. | 2025-05-19 14:35:41.770092

--- Contents of table: analytics ---
id | type | chat_history_id | created_at
1 | like | 1 